In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('./r101.csv')
XCOORD, YCOORD = data['XCOORD'].iloc, data['YCOORD'].iloc
READYTIME, DUEDATE = data['READYTIME'].iloc, data['DUEDATE'].iloc
SERVICETIME = data['SERVICETIME'].iloc
print(data)

     CUSTNO  XCOORD  YCOORD  DEMAND  READYTIME  DUEDATE  SERVICETIME
0         0      35      35       0          0      230            0
1         1      41      49      10        161      171           10
2         2      35      17       7         50       60           10
3         3      55      45      13        116      126           10
4         4      55      20      19        149      159           10
..      ...     ...     ...     ...        ...      ...          ...
97       97      25      21      12        133      143           10
98       98      19      21      10         58       68           10
99       99      20      26       9         83       93           10
100     100      18      18      17        185      195           10
101     101      35      35       0          0      230            0

[102 rows x 7 columns]


In [3]:
def Connect(former, latter):
    connected = former[:-1] + latter[1:]
    return connected

def Dist(i, j):
    dist = np.sqrt((XCOORD[i]-XCOORD[j])**2 + (YCOORD[i]-YCOORD[j])**2)
    dist = round(dist, 2)
    return dist

def TotalDist(route):
    total_dist = 0
    for point in range(len(route)-1):
        i, j = route[point], route[point+1]
        dist = Dist(i,j)
        total_dist += dist
    total_dist = round(total_dist, 2)
    return total_dist

def LastEndService(route):
    begin_j = 0
    for point in range(len(route)-1):
        begin_i = begin_j
        i, j = route[point], route[point+1]
        begin_j = max(READYTIME[j], begin_i+SERVICETIME[i]+Dist(i,j))
    end_service = begin_i+SERVICETIME[route[-2]]
    return end_service

In [4]:
current = initial = [[0, i, 101] for i in range(1, 101)]
while True:
    max_saving, max_wait = 0, 20
    for r1 in current:
        for r2 in current[current.index(r1)+1:]:
            # Check former or latter
            if (LastEndService(r1) + Dist(r1[-2], r2[1])) <= DUEDATE[r2[1]]:
                froute, lroute = r1, r2
            elif (LastEndService(r2) + Dist(r2[-2], r1[1])) <= DUEDATE[r1[1]]:
                froute, lroute = r2, r1
            else:
                continue
            # Waiting time threshold
            arrive_next = LastEndService(froute) + Dist(froute[-2], lroute[1])
            if READYTIME[lroute[1]] - arrive_next > max_wait:
                continue
            # Calculate saving
            saving = round(TotalDist(r1) + TotalDist(r2) - TotalDist(Connect(froute, lroute)), 2)
            # Record max_saving
            if saving > max_saving:
                max_saving = saving
                max_froute = froute
                max_lroute = lroute
    # Termination condition
    if max_saving == 0:
        break
    current = [sublist for sublist in current if (sublist != max_froute and sublist != max_lroute)]
    current.append(Connect(max_froute, max_lroute))
# Result
final_status, final_dist = current, 0
for route in final_status:
    final_dist += TotalDist(route)
print(f'Num of Vehicle: {len(final_status)}, Distance: {round(final_dist, 2)}')

Num of Vehicle: 22, Distance: 1863.56
